# Spatial fidelity template
Infers a spatial fiedlity map for every individual from a ```.mymridon``` experiment file and saves it as a csv, which can be further analyzed in the optinal part or exported into other software such as RStudio.  
This notebook uses the following:
* the py-myrmidon library ([Documentation](https://formicidae-tracker.github.io/myrmidon/latest/))
* scipy spatial library ([Documentation](https://docs.scipy.org/doc/scipy/reference/spatial.html))

In [ ]:
from datetime import datetime, timedelta  # For convenient handling of time and date

import numpy as np  # Fundamental math library in python.
import pandas as pd  # To create a pandas dataframe, an equivalent to an R dataframe
import py_fort_myrmidon as fm
from scipy.spatial import KDTree

## Calculating spatial fidelity

The code here is the same as in the original code written by Matthias. For convenience sake the different components have been converted into functions that can be used to run over multiple fort-studio experiments

### Defining the grid for the spatial fidelity

A spatial tesselation of hexagons is used to define the sites where the observations of the individuals are counted. A nearest neighbour tree (KDTree) is used with the centers of the hexgons mentioned. For a given set of coordinates, a KDTree can be used to efficiently find the seed that is closest to the input coordinates, here the waypoints of the trajectory. The hexagonal boundaries are not defined explicitly. Simply by placing the hexagonal centers accordingly (see ```method ``` variable), the resulting pattern will be hexagonal or squared. The code in the following cell computes the hexagon centers that seed the KDTree.  
This part should probably only be edited w.r.t. the method and the internal radius.

In [ ]:
def define_kdtree(method, xlim, ylim, r_int):
    if method == "vertical":
        dx = 2 * r_int
        dy = np.sqrt(3) * r_int
    elif method == "horizontal":
        dx = np.sqrt(3) * r_int
        dy = 2 * r_int
    elif method == "square":
        dx = 2 * r_int
        dy = 2 * r_int

    n_x = np.diff(xlim) // dx + 1
    n_y = np.diff(ylim) // dy + 1
    osx = n_x * dx - np.diff(xlim)
    osy = n_y * dy - np.diff(ylim)
    x = np.arange(start=xlim[0] - osx / 2, step=dx, stop=n_x * dx + osx / 2)
    y = np.arange(start=ylim[0] - osy / 2, step=dy, stop=n_y * dy + osy / 2)
    xm, ym = np.meshgrid(x, y)
    if method == "vertical":
        xm[::2] = xm[::2] + r_int
    elif method == "horizontal":
        ym[:, 1::2] = ym[:, 1::2] + r_int

    sites = np.stack([xm.flatten(), ym.flatten()], axis=1)
    kdtree = KDTree(sites)
    return kdtree, sites

### Function to calculate presence in the grid

This function takes the counts of each antID in each of the grid cells that were calculated using the kdtree function

In [ ]:
def calc_counts_kdtree(
    exp, t_start, t_end, kdtree, counts, idxmap, counts_cutoff, col_phase
):
    """Function to calculate normalised counts of ants in each zone

    Args:
        exp (fort-myrmidon experiment file): Fort myrmidon experiment file
        t_start (datetime): Start time in local time with timezone set to None
        t_end (datetime): End time in local time with timezone set to None
        kdtree (kdtree object): kdtree object ontained from define_kdtree function
        counts (Numpy Array): A numpy array with dimentsions number of ants x number of zones
        idxmap (dict): Dictionary mapping ant ids to indices in the counts array
        counts_cutoff (int): cut-off value to threshold the counts
        col_phase (string): String which contains the colony and phase information
    """
    # Convert to fm.Time object
    t_st = fm.Time(t_start)
    t_en = fm.Time(t_end)
    # Extract ant trajectories
    trajectories = fm.Query.ComputeAntTrajectories(
        exp,
        start=t_st,
        end=t_en,
    )
    # Obtain the zone indices for each coordinate in each trajectory
    for t in trajectories:  # Loop over all the trajectories
        dist, zone_indices = kdtree.query(
            t.Positions[:, 1:3]
        )  # Obtain the zone indices for each (X,Y) coordinate in the trajectory
        ind, cts = np.unique(
            zone_indices, return_counts=True
        )  # Count the number of times each zone is present
        counts[t.Space][idxmap[t.Ant], ind] += cts  # Add the counts to the counts array
    # Normalize the counts
    for s in exp.Spaces:  # Loop over the experimental spaces
        row_sums = counts[s].sum(axis=1)[
            :, np.newaxis
        ]  # Sum the counts for each individual
        counts[s] = np.divide(
            counts[s], row_sums, out=np.zeros_like(counts[s]), where=row_sums != 0
        )  # Normalize per individual, removing individuals which have less than counts_cutoff
        # Output the counts to a csv file
        df = pd.DataFrame(data=counts[s], index=exp.Ants)  # Create a pandas dataframe
        df.index.name = "AntID"  # Name the index column
        f_count = "NormCounts_{}_Space{}_{}_{}_{}.csv".format(
            col_phase,
            s,
            exp.Spaces[s].Name,
            t_start.strftime("%Y%m%d-%H%M%S"),
            t_end.strftime("%Y%m%d-%H%M%S"),
        )  # Name for the output file
        print(f_count, " done")
        df.to_csv(f_count)

### Function to run the spatial presence calculation over whole experiments

This is a wrapper function for `calc_counts_kdtree` which runs the function over multiple time ranges

In [ ]:
def calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
):
    # Open myrmidon file
    exp = fm.Experiment.Open(myrmidon_path)
    # Get frames from first second of experiment tracking
    identifiedFrames = fm.Query.IdentifyFrames(
        exp,
        start=fm.Time(exp_start),
        end=fm.Time(exp_start).Add(fm.Duration().Parse("1s")),
    )
    # Obtain x and y limits fo frame
    xlim = [0, identifiedFrames[0].Width]
    ylim = [0, identifiedFrames[0].Height]
    # get average ant size to calculate the bin size
    # ant_radius = []
    # for ant in exp.Ants:
    #     ant_radius.append(fm.Query.ComputeMeasurementFor(experiment=exp, antID=ant, measurementTypeID=1)[0].LengthPixel)
    # r_int = np.mean(ant_radius)
    r_int = 300  # Using a general value of 300 to account for cases where the fm.Query.ComputeMeasurementFor throws an error
    # Obtain the kdtree
    kdtree, sites = define_kdtree("vertical", xlim, ylim, r_int)
    # Create a dictionary for holding counts data
    # counts_cutoff = 0 # For hard coding a count cutoff inside the function
    # Create a dictionary with an array full of zeros and shape (number individuals x number sites) for each Space
    counts = {}
    for s in exp.Spaces:
        counts[s] = np.zeros((len(exp.Ants), len(sites)))
    idxmap = dict(
        zip(exp.Ants, range(len(exp.Ants)))
    )  # Maps the ant id to the matrix index
    # Run function to calculate counts over each space for each combination of start and end time
    [
        calc_counts_kdtree(
            exp, t_start, t_end, kdtree, counts, idxmap, counts_cutoff, col_phase
        )
        for t_start, t_end, col_phase in zip(
            start_time_list, end_time_list, col_phase_list
        )
    ]

## Colony Cfel42

In [ ]:
# Colony 42
myrmidon_path = "/media/ebiag/Ebi-2/Woundcare Experiment1/Cfell_wound_col42.myrmidon"
exp_start = datetime(2022, 4, 27, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2022, 5, 1, 15, 54).astimezone(tz=None),
    datetime(2022, 5, 2, 16, 3).astimezone(tz=None),
    datetime(2022, 5, 3, 15, 53).astimezone(tz=None),
    datetime(2022, 5, 4, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 5, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 6, 15, 55).astimezone(tz=None),
    datetime(2022, 5, 2, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 3, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 4, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 5, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 6, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 7, 9, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel42_"
phase = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [ ]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)

## Colony Cfel1

In [ ]:
# Colony 1
myrmidon_path = "/media/ebiag/Ebi-2/Woundcare Experiment2/woundcare_cfell1_T2.myrmidon"
exp_start = datetime(2022, 5, 31, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2022, 6, 4, 14, 48).astimezone(tz=None),
    datetime(2022, 6, 5, 14, 57).astimezone(tz=None),
    datetime(2022, 6, 6, 14, 30).astimezone(tz=None),
    datetime(2022, 6, 7, 14, 49).astimezone(tz=None),
    datetime(2022, 6, 8, 14, 43).astimezone(tz=None),
    datetime(2022, 6, 9, 15, 5).astimezone(tz=None),
    datetime(2022, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 6, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 7, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 8, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 9, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 10, 8, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel1_"
phase = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [ ]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)

## Colony Cfel54

In [ ]:
myrmidon_path = "/media/ebiag/Ebi-2/Woundcare Experiment3/woundcare_cfell54_T3.myrmidon"
exp_start = datetime(2022, 6, 15, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2022, 6, 19, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 20, 14, 35).astimezone(tz=None),
    datetime(2022, 6, 21, 14, 21).astimezone(tz=None),
    datetime(2022, 6, 22, 14, 28).astimezone(tz=None),
    datetime(2022, 6, 23, 14, 14).astimezone(tz=None),
    datetime(2022, 6, 24, 14, 31).astimezone(tz=None),
    datetime(2022, 6, 20, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 21, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 22, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 23, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 24, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 25, 8, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel54_"
phase = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [ ]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)

## Colony Cfel 55

In [ ]:
# Colony 55
myrmidon_path = "/media/ebiag/Ebi-3/InfectionExp_Cfel55/InfectionExpCol55.myrmidon"
exp_start = datetime(2023, 4, 14, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2023, 4, 18, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 20, 15, 45).astimezone(tz=None),
    datetime(2023, 4, 21, 14, 48).astimezone(tz=None),
    datetime(2023, 4, 22, 14, 17).astimezone(tz=None),
    datetime(2023, 4, 23, 14, 0).astimezone(tz=None),
    datetime(2023, 4, 24, 14, 54).astimezone(tz=None),
    datetime(2023, 4, 20, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 21, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 22, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 23, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 24, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 25, 8, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel55_"
phase = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [ ]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)

## Colony Cfel 13

In [ ]:
# Colony 13
myrmidon_path = "/media/ebiag/Ebi-3/InfectionExp_Cfel13/InfectionExp_Cfel13.myrmidon"
exp_start = datetime(2023, 4, 19, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2023, 4, 23, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 24, 15, 29).astimezone(tz=None),
    datetime(2023, 4, 25, 14, 19).astimezone(tz=None),
    datetime(2023, 4, 26, 15, 3).astimezone(tz=None),
    datetime(2023, 4, 27, 16, 43).astimezone(tz=None),
    datetime(2023, 4, 28, 14, 27).astimezone(tz=None),
    datetime(2023, 4, 24, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 25, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 26, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 27, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 28, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 29, 8, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel13_"
phase = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [ ]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)

## Colony Cfel 64

In [ ]:
# Colony 64
myrmidon_path = "/media/ebiag/Ebi-1/InfectionExp_Cfel64/InfectionExpCol64.myrmidon"
exp_start = datetime(2023, 5, 27, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2023, 5, 31, 15, 5).astimezone(tz=None),
    datetime(2023, 6, 1, 15, 51).astimezone(tz=None),
    datetime(2023, 6, 2, 14, 44).astimezone(tz=None),
    datetime(2023, 6, 3, 14, 50).astimezone(tz=None),
    datetime(2023, 6, 4, 14, 43).astimezone(tz=None),
    datetime(2023, 6, 5, 14, 52).astimezone(tz=None),
    datetime(2023, 6, 1, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 2, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 3, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 4, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 6, 8, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel64_"
phase = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [ ]:
c64_list = calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)